##### Copyright 2023 The IREE Authors

In [1]:
#@title Licensed under the Apache License v2.0 with LLVM Exceptions.
# See https://llvm.org/LICENSE.txt for license information.
# SPDX-License-Identifier: Apache-2.0 WITH LLVM-exception

# Dynamic Shapes

This notebook

1. Creates a PyTorch program with dynamic shapes using [SHARK-Turbine](https://github.com/nod-ai/SHARK-Turbine)'s advanced AOT toolkit
2. Compiles the program to an IREE VM bytecode module
3. Tests running the compiled VM module using IREE's runtime
4. Downloads compilation artifacts for use with the native (C API) sample application

In [1]:
#@title General setup

import os
import tempfile

ARTIFACTS_DIR = os.path.join(tempfile.gettempdir(), "iree", "colab_artifacts")
os.makedirs(ARTIFACTS_DIR, exist_ok=True)
print(f"Using artifacts directory '{ARTIFACTS_DIR}'")

Using artifacts directory '/tmp/iree/colab_artifacts'


In [2]:
%%capture
#@title Uninstall existing packages
#   This avoids some warnings when installing specific PyTorch packages below.
!python -m pip uninstall -y fastai torchaudio torchdata torchtext torchvision

In [3]:
#@title Install SHARK-Turbine

# Limit cell height.
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

!python -m pip install shark-turbine

<IPython.core.display.Javascript object>

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.5 MB/s eta 0:00:00
  Created wheel for shark-turbine: filename=shark_turbine-0.9.1.dev3-py3-none-any.whl size=70102 sha256=507dec827b9a2eea18f47c6ebdc84347c9956b8f2e0b186d3107a006e0742d81
  Stored in directory: /root/.cache/pip/wheels/e9/78/0f/88c9d8224ef1550fe00b18a014eab5121f26264e2261f31926
Successfully built shark-turbine


In [4]:
#@title Report version information
!echo "Installed SHARK-Turbine, $(python -m pip show shark_turbine | grep Version)"

!echo -e "\nInstalled IREE, compiler version information:"
!iree-compile --version

import torch
print("\nInstalled PyTorch, version:", torch.__version__)

Installed SHARK-Turbine, Version: 0.9.1.dev3

Installed IREE, compiler version information:
IREE (https://iree.dev):
  IREE compiler version 20231004.665 @ bb51f6f1a1b4ee619fb09a7396f449dadb211447
  LLVM version 18.0.0git
  Optimized build

Installed PyTorch, version: 2.1.0+cu118


## Create a program using PyTorch + SHARK-Turbine

NOTE: as in other domains, providing more information to a compiler allows it
to generate more efficient code. As a general rule, the slowest varying
dimensions of program data like batch index or timestep are safer to treat as
dynamic than faster varying dimensions like image x/y/channel. See
[this paper](https://arxiv.org/pdf/2006.03031.pdf) for a discussion of the
challenges imposed by dynamic shapes and one project's approach to addressing
them.

In [5]:
#@title Define a sample `shark_turbine.aot.CompiledModule` using dynamic shapes

import shark_turbine.aot as aot

class DynamicShapesModule(aot.CompiledModule, export_name="module"):
  # reduce_sum_1d (dynamic input size, static output size)
  #   tensor<?xi32> -> tensor<i32>
  #   e.g. [1, 2, 3] -> 6
  def reduce_sum_1d(self, values=aot.AbstractTensor(None, dtype=torch.int32)):
    return self.compute_reduce_sum_1d(values)

  @aot.jittable
  def compute_reduce_sum_1d(values):
    return torch.sum(values, dtype=torch.int32)

  # reduce_sum_2d (partially dynamic input size, static output size)
  #   tensor<?x3xi32> -> tensor<3xi32>
  #   e.g. [[1, 2, 3], [10, 20, 30]] -> [11, 22, 33]
  def reduce_sum_2d(self, values=aot.AbstractTensor(None, 3, dtype=torch.int32)):
    return self.compute_reduce_sum_2d(values)

  @aot.jittable
  def compute_reduce_sum_2d(values):
    return torch.sum(values, 0, dtype=torch.int32)

  # add_one (dynamic input size, dynamic output size)
  #   tensor<?xi32>) -> tensor<?xi32>
  #   e.g. [1, 2, 3] -> [2, 3, 4]
  def add_one(self, values=aot.AbstractTensor(None, dtype=torch.int32)):
    return self.compute_add_one(values)

  @aot.jittable
  def compute_add_one(values):
    return values + 1

In [6]:
from iree.compiler.ir import Context

# Import into MLIR and save to disk.
dynamic_shapes_instance = DynamicShapesModule(context=Context())
imported_mlir_path = os.path.join(ARTIFACTS_DIR, "dynamic_shapes.mlir")
aot.CompiledModule.save_mlir(dynamic_shapes_instance, imported_mlir_path)
print(f"Wrote MLIR to path '{imported_mlir_path}'")

# Inspect the IR.
# Note the question marks for dynamic shapes in types, like `tensor<?xi32>`.
print("\nDynamic Shapes MLIR:")
!cat {imported_mlir_path}

Wrote MLIR to path '/tmp/iree/colab_artifacts/dynamic_shapes.mlir'

Dynamic Shapes MLIR:
#map = affine_map<(d0) -> (d0)>
#map1 = affine_map<(d0) -> ()>
#map2 = affine_map<(d0, d1) -> (d0, d1)>
#map3 = affine_map<(d0, d1) -> (d1)>
module @module {
  func.func @reduce_sum_1d(%arg0: tensor<?xi32>) -> tensor<i32> attributes {torch.args_schema = "[1, {\22type\22: \22builtins.tuple\22, \22context\22: \22null\22, \22children_spec\22: [{\22type\22: \22builtins.list\22, \22context\22: \22null\22, \22children_spec\22: [{\22type\22: null, \22context\22: null, \22children_spec\22: []}]}, {\22type\22: \22builtins.dict\22, \22context\22: \22[]\22, \22children_spec\22: []}]}]", torch.return_schema = "[1, {\22type\22: null, \22context\22: null, \22children_spec\22: []}]"} {
    %0 = call @compute_reduce_sum_1d(%arg0) : (tensor<?xi32>) -> tensor<i32>
    return %0 : tensor<i32>
  }
  func.func private @compute_reduce_sum_1d(%arg0: tensor<?xi32>) -> tensor<i32> {
    %c0_i32 = arith.constant 0 : i32
   

## Test the imported program

_Note: you can stop after each step and use intermediate outputs with other tools outside of Colab._

_See the [README](https://github.com/openxla/iree/tree/main/samples/dynamic_shapes#instructions) for more details and example command line instructions._

* _The "imported MLIR" (above) can be used by IREE's generic compiler tools_
* _The "binary" can be saved and used by runtime applications_

_The specific point at which you switch from Python to native tools will depend on your project._

In [7]:
# Export and compile.
exported_output = aot.export(DynamicShapesModule)

# Compile to a file on disk for usage outside of Python.
flatbuffer_path = os.path.join(ARTIFACTS_DIR, "dynamic_shapes_cpu.vmfb")
exported_output.compile(save_to=flatbuffer_path)
print(f"Wrote compiled program to path '{flatbuffer_path}'")

# Compile into memory for testing.
binary = exported_output.compile(save_to=None)

Wrote compiled program to path '/tmp/iree/colab_artifacts/dynamic_shapes_cpu.vmfb'


In [8]:
import iree.runtime as ireert
import numpy as np

# Use the IREE runtime API to test the compiled program.
config = ireert.Config("local-task")
vm_module = ireert.load_vm_module(
    ireert.VmModule.wrap_buffer(config.vm_instance, binary.map_memory()),
    config,
)

print(vm_module.reduce_sum_1d(np.array([1, 10, 100], dtype=np.int32)).to_host())
print(vm_module.reduce_sum_2d(np.array([[1, 2, 3], [10, 20, 30]], dtype=np.int32)).to_host())
print(vm_module.reduce_sum_2d(np.array([[1, 2, 3], [10, 20, 30], [100, 200, 300]], dtype=np.int32)).to_host())
print(vm_module.add_one(np.array([1, 10, 100], dtype=np.int32)).to_host())

111
[11 22 33]
[111 222 333]
[  2  11 101]


## Download compilation artifacts

In [9]:
ARTIFACTS_ZIP = "/tmp/dynamic_shapes_colab_artifacts.zip"

print(f"Zipping '{ARTIFACTS_DIR}' to '{ARTIFACTS_ZIP}' for download...")
!cd {ARTIFACTS_DIR} && zip -r {ARTIFACTS_ZIP} .

# Note: you can also download files using Colab's file explorer
try:
  from google.colab import files
  print("Downloading the artifacts zip file...")
  files.download(ARTIFACTS_ZIP)
except ImportError:
  print("Missing google_colab Python package, can't download files")

Zipping '/tmp/iree/colab_artifacts' to '/tmp/dynamic_shapes_colab_artifacts.zip' for download...
  adding: dynamic_shapes_cpu.vmfb (deflated 66%)
  adding: dynamic_shapes.mlir (deflated 82%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>